In [1]:
# init imports

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import (Dense,
                                     BatchNormalization,
                                     LeakyReLU,
                                     Reshape,
                                     Conv2DTranspose,
                                     Conv2D,
                                     Dropout,
                                     Flatten)

import matplotlib.pyplot as plt
from PIL import Image

In [2]:
BUFFER_SIZE = 65_536
BATCH_SIZE = 8
DATA_ID = '_5s_20hz'
dataset_path = "data/dataset" + DATA_ID + "/" + "train_feats" + DATA_ID + ".npy"

with tf.device('/CPU:0'):
    # load in dataset
    train_dataset_npy = np.load(dataset_path, allow_pickle=True)
    print ('dataset.shape: ', train_dataset_npy.shape)
    # print ('train_dataset: ', train_dataset)
    # normalize images to [-1, 1]
    # norm_train_dataset = (train_dataset_npy / 127.5) - 1.0
    # print ('normalized train_dataset: ', train_dataset)
    # batch and shuffle data
    train_dataset_tensor = tf.data.Dataset.from_tensor_slices(train_dataset_npy)
    train_dataset_tensor = train_dataset_tensor.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    train_dataset_tensor = train_dataset_tensor.prefetch(tf.data.AUTOTUNE)
    
    print ('dataset-tensor.shape: ', train_dataset_tensor.cardinality())

In [3]:
from ipynb.fs.full.dataset_creation import show_midi_plot

def view_midi(np_array):
    np_array = np_array.reshape(np_array.shape[0]*np_array.shape[1], np_array.shape[2])
    show_midi_plot(np_array)